In [1]:
# Dependencies
import requests
from config import api_key, CLIENT_ID, CLIENT_SECRET
import json
import pprint as pp
import pandas as pd
import time
import lyricsgenius

In [2]:
# Request Access Token for Spotify API calls.
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [3]:
# Build query URL
# Access the playlist here: https://open.spotify.com/playlist/1Jczb1SckWxYiqrIzv0jsl?si=560c28541ccb42f5

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

url = "https://api.spotify.com/v1/"

play_id = "1Jczb1SckWxYiqrIzv0jsl?si=560c28541ccb42f5"

query_url = url+'playlists/'+play_id+'tracks'

In [4]:
# Get playlist data and initialize arrays for DataFrame assembly.
json_req = requests.get(query_url, headers=headers)
response = json_req.json()

track_ids = []
track_names = []
artist_names = []
acousticness = []
danceability = []
energy = []
instrumentalness = []
liveness = []
loudness = []
speechiness = []
tempo = []
valence = []

#Assign each genre as listed in the playlist.
genres = []
for j in range (1,9):
    for i in range(1,16):
        genres.append(j)

In [5]:
#Append track ID and related song data, then make separate API call to find the track's audio features.
first_part = True
while True:
    #First 100 songs in the playlist.
    if first_part == True:
        for item in response['tracks']['items']:
            track_names.append(item['track']['name'])
            track_ids.append(item['track']['id']) 
            artist_names.append(item['track']['album']['artists'][0]['name'])
            #Make new API call on track's audio features based on each track's ID.
            audio_query_url = url+'audio-features/'+item['track']['id']
            json_req = requests.get(audio_query_url, headers=headers)
            audio_response = json_req.json()
            acousticness.append(audio_response['acousticness'])
            danceability.append(audio_response['danceability'])
            energy.append(audio_response['energy'])
            instrumentalness.append(audio_response['instrumentalness'])
            liveness.append(audio_response['liveness'])
            loudness.append(audio_response['loudness'])
            speechiness.append(audio_response['speechiness'])
            tempo.append(audio_response['tempo'])
            valence.append(audio_response['valence'])
        #Switch to second half of API request (Index max of 100 tracks in playlist per request)
        audio_response = response['tracks']['next']
        first_part = False
        time.sleep(3)
    else:
        req = requests.get(audio_response, headers=headers)
        data = req.json()
        for item in data['items']:
            track_names.append(item['track']['name'])
            track_ids.append(item['track']['id']) 
            artist_names.append(item['track']['album']['artists'][0]['name'])
            #Make new API call on track's audio features based on each track's ID.
            audio_query_url = url+'audio-features/'+item['track']['id']
            json_req = requests.get(audio_query_url, headers=headers)
            audio_response = json_req.json()
            acousticness.append(audio_response['acousticness'])
            danceability.append(audio_response['danceability'])
            energy.append(audio_response['energy'])
            instrumentalness.append(audio_response['instrumentalness'])
            liveness.append(audio_response['liveness'])
            loudness.append(audio_response['loudness'])
            speechiness.append(audio_response['speechiness'])
            tempo.append(audio_response['tempo'])
            valence.append(audio_response['valence'])
        break

#Assemble the DataFrame.
audio_df = pd.DataFrame({ 'Track Name':track_names, 'Artist':artist_names, 'Track ID':track_ids, 'Genres':genres, 'Acousticness':acousticness,
                  'Danceability':danceability, 'Energy':energy, 'Instrumentalness':instrumentalness, 'Liveness':liveness,'Loudness':loudness,
                  'Speechiness':speechiness, 'Tempo':tempo, 'Valence':valence})



In [6]:
audio_df.head(20)

,Track Name,Artist,Track ID,Genres,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence
0,The Real Slim Shady,Eminem,3yfqSUWxFvZELEM4PmlwIR,1,0.030200,0.949,0.661,0.000000,0.0454,-4.244,0.0572,104.504,0.760
1,MotorSport,Migos,4wFjTWCunQFKtukqrNijEt,1,0.030500,0.904,0.518,0.000000,0.3250,-5.320,0.1830,137.996,0.188
2,In Da Club,50 Cent,5Mcvgt2GbxxCp6cz3Kfjey,1,0.325000,0.906,0.663,0.000000,0.0883,-3.955,0.3550,90.042,0.847
3,Gangsta's Paradise,Coolio,1DIXPcTDzTj8ZMHt3PDt8p,1,0.065500,0.647,0.514,0.000000,0.3980,-10.050,0.0593,79.975,0.386
4,rockstar,Post Malone,19bX4zmC2zDben2ldhoRB3,1,0.145000,0.594,0.507,0.000871,0.1320,-6.141,0.0816,159.729,0.137
5,HUMBLE.,Kendrick Lamar,131OLY5J8XyfGuSjXRiTRM,1,0.000253,0.911,0.566,0.000075,0.0956,-6.703,0.1350,150.039,0.374
6,Black and Yellow,Wiz Khalifa,3VP78k3jzm0Q5OM08E383k,1,0.061900,0.685,0.832,0.000000,0.2730,-4.548,0.0595,164.070,0.574
7,WAP (feat. Megan Thee Stallion),Cardi B,051wt8AyLFgYnVuberd3vO,1,0.027200,0.934,0.443,0.000000,0.0889,-7.541,0.4100,133.026,0.359
8,Panini,Lil Nas X,6fTt0CH2t0mdeB2N9XFG5r,1,0.342000,0.703,0.594,0.000000,0.1230,-6.146,0.0752,153.848,0.475
9,Reflections,Bliss n Eso,5Wf0EKbK4AB6xtSbsqL4r2,1,0.725000,0.497,0.605,0.000000,0.2380,-8.625,0.0468,77.811,0.824


In [7]:
#Append lyric information from Genius API.
#Import ID and Secret.
CLIENT_ID = 'mVxsgubdxvPwivQOYFWt5FCskAPVyDRl7ZQnt4PPvlBaPHfbTATSqoqt2KKVjji1'
CLIENT_SECRET = 'u6bH4rznsJK0A80jHqIkJ3I6XxJr6IbtY0EX7T4aDDre5wCREj9ywuA5khlbyNM9AeKLoSFcYjYLA6vCTOKhPg'
genius = lyricsgenius.Genius('3Ee9v1MsjFxlAwwA0OkZWNJEHWRBhYWqpp5TwwWk_UX70_P36TH7StPKeBLt88EI')

In [8]:
#For-loop that iterates through first column of audio_df, searches for matching lyrics
lyrics = []
failed = []
i = 0
for index, row in audio_df.iterrows():
    song_title = row['Track Name']
    try:
        songs = genius.search_song(song_title)
        lyrics.append(songs.lyrics)
        time.sleep(2)
    except:
        try:
            songs = genius.search_song(song_title)
            lyrics.append(songs.lyrics)
            time.sleep(2)
        except:
            lyrics.append(0)
            print('Error retrieving lyrics. Moving on to the next track...')
            failed.append(song_title)
            time.sleep(5)
            pass


Searching for "The Real Slim Shady"...
Done.
Searching for "MotorSport"...
Done.
Searching for "In Da Club"...
Searching for "In Da Club"...
Error retrieving lyrics. Retrying on next track...
Searching for "Gangsta's Paradise"...
Done.
Searching for "rockstar"...
Searching for "rockstar"...
Error retrieving lyrics. Retrying on next track...
Searching for "HUMBLE."...
Done.
Searching for "Black and Yellow"...
Done.
Searching for "WAP (feat. Megan Thee Stallion)"...
Done.
Searching for "Panini"...
Searching for "Panini"...
Done.
Searching for "Reflections"...
Done.
Searching for "Push It"...
Searching for "Push It"...
Error retrieving lyrics. Retrying on next track...
Searching for "Bodak Yellow"...
Done.
Searching for "Super Bass"...
Done.
Searching for "Paper Planes"...
Searching for "Paper Planes"...
Error retrieving lyrics. Retrying on next track...
Searching for "Black Widow"...
Done.
Searching for "Rapper's Delight"...
Searching for "Rapper's Delight"...
Error retrieving lyrics. Re

In [11]:
print("Found lyrics:", len(lyrics)-len(failed), "Failed:", len(failed))

Found lyrics: 95 Failed: 25


In [12]:
#Append lyric information to df.
lyrics_df = audio_df.copy()
lyrics_df['Lyrics'] = lyrics
lyrics_df.head()

,Track Name,Artist,Track ID,Genres,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Lyrics
0,The Real Slim Shady,Eminem,3yfqSUWxFvZELEM4PmlwIR,1,0.0302,0.949,0.661,0.000000,0.0454,-4.244,0.0572,104.504,0.760,459 ContributorsTranslationsEspañolPortuguêsPo...
1,MotorSport,Migos,4wFjTWCunQFKtukqrNijEt,1,0.0305,0.904,0.518,0.000000,0.3250,-5.320,0.1830,137.996,0.188,428 ContributorsTranslationsPortuguêsMotorSpor...
2,In Da Club,50 Cent,5Mcvgt2GbxxCp6cz3Kfjey,1,0.3250,0.906,0.663,0.000000,0.0883,-3.955,0.3550,90.042,0.847,0
3,Gangsta's Paradise,Coolio,1DIXPcTDzTj8ZMHt3PDt8p,1,0.0655,0.647,0.514,0.000000,0.3980,-10.050,0.0593,79.975,0.386,234 ContributorsTranslationsMagyarGangsta’s Pa...
4,rockstar,Post Malone,19bX4zmC2zDben2ldhoRB3,1,0.1450,0.594,0.507,0.000871,0.1320,-6.141,0.0816,159.729,0.137,0


In [15]:
# Try again for rows which returned '0'. (Re-run this section until all lyrics implemented.)
lyrics = []
failed = []
i = 0
for row in lyrics_df.index:
    if lyrics_df['Lyrics'][row] == 0:
        song_title = lyrics_df['Track Name'][row]
        try:
            songs = genius.search_song(song_title)
            lyrics_df['Lyrics'][row] = songs.lyrics
            time.sleep(2)
        except:
            try:
                songs = genius.search_song(song_title)
                lyrics_df['Lyrics'][row] = songs.lyrics
                time.sleep(2)
            except:
                lyrics.append(0)
                print('Error retrieving lyrics. Skipping to next track...')
                failed.append(song_title)
                time.sleep(5)
                pass
print("Found lyrics:", len(lyrics), "Failed:", len(failed))

Searching for "Gin and Juice"...
Done.


C:\Users\Alex\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


Found lyrics: 0 Failed: 0


In [30]:
pp.pprint(weird_df.iloc[51,13])

('78 ContributorsLegacy Lyrics[Songtext zu „Legacy“]\n'
 '\n'
 '[Part 1]\n'
 'Ich bin siebzehn, um mich rum nur resignierte Zukunftspläne\n'
 'Meine Homies Teil einer deprimierten Jugendszene\n'
 'Frust statt Karriere, unter kaputter Laterne\n'
 'Durch bisschen Wodka kriegt die Bushaltestelle Clubatmosphäre\n'
 'Ah, statt ausharren und warten\n'
 "Musst du'n Traum haben, dich aufmachen, aufraffen und starten\n"
 "Stell dich dem Ziel, 'ne Perspektive ist nix\n"
 'Was die Gesellschaft dir gibt, sondern was du selbst in dir siehst\n'
 "Mir droht der Schulrausschmiss '05 im Abijahr, mach' Money klar\n"
 'Als Grasdealer, die andern planen Urlaubstrips\n'
 'Wie Australienbackpackreisen\n'
 'Ich kann mir grad vom Ersparten in Maastricht von Marokkanern paar '
 'Marihuanapackets leisten\n'
 "Beginn' die Ware zu verteil'n, mit oberflächlichem Wissen\n"
 'Von Dopegeschäften an Homes an den Großstadtecken zu ticken\n'
 "Will bloß was Besseres mitten in 'ner Zeit, wo's kein'n Lohn gab\n"
 "Ich bro

In [31]:
#lyrics_df.head()
# Set lyrics to '0' for songs which were unable to come up with correct or accurate lyrics.
weirdsongs = [51,53,60,98,118]
weird_df = lyrics_df.copy()
for row in weird_df.index:
    if row in weirdsongs:
         weird_df['Lyrics'][row] = 0
pp.pprint(weird_df.iloc[51,13])

0


C:\Users\Alex\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [79]:
clean_df = weird_df.copy()
clean_df.to_json("clean_lyrics_df.json")
#Above steps already completed in 'lyrics_df.json'
# To import from this checkpoint, deleted comment and run: lyrics_df = pd.read_json("clean_lyrics_df")
#df.head()

'459 ContributorsTranslationsEspañolPortuguêsPolskiItalianoDeutschFrançaisTürkçeThe Real Slim Shady Lyrics[Intro]\nMay I have your attention, please?\nMay I have your attention, please?\nWill the real Slim Shady please stand up?\nI repeat, will the real Slim Shady please stand up?\nWe\'re gonna have a problem here...\n\n[Verse 1]\nY\'all act like you never seen a white person before\nJaws all on the floor like Pam like Tommy just burst in the door\nAnd started whoopin\' her ass worse than before\nThey first were divorced, throwin\' her over furniture (Agh!)\nIt\'s the return of the "Oh, wait, no way, you\'re kidding\nHe didn\'t just say what I think he did, did he?"\nAnd Dr. Dre said… nothing, you idiots!\nDr. Dre\'s dead, he\'s locked in my basement (Ha ha!)\nFeminist women love Eminem\n"Chicka, chicka, chicka, Slim Shady, I\'m sick of him\nLook at him, walkin\' around, grabbin\' his you-know-what\nFlippin\' the you-know-who," "Yeah, but he\'s so cute though"\nYeah, I probably got a c

In [104]:
# Cleaning lyrics (removing Chorus, Verse, Bridge tags etc. and embed statements at beginning and end of songs)
superclean_df = clean_df.copy()
superclean_df['Lyrics'] = superclean_df['Lyrics'].str.replace(r"\d.*\sLyrics","", regex=True)
superclean_df['Lyrics'] = superclean_df['Lyrics'].str.replace(r"\d.*Embed"," ", regex=True)
superclean_df['Lyrics'] = superclean_df['Lyrics'].str.replace(r"\[.*\]"," ", regex=True)
superclean_df['Lyrics'] = superclean_df["Lyrics"].str.replace(r"\d*Embed",' ',regex=True)
superclean_df['Lyrics'] = superclean_df['Lyrics'].str.replace(r"\\n",' ', regex=True)
superclean_df['Lyrics'] = superclean_df['Lyrics'].str.replace(r"\\",'', regex=True)
superclean_df['Lyrics'] = superclean_df['Lyrics'].str.replace(r"\(|\)|\!|\?|\.|,",'', regex=True)
superclean_df.iloc[0,13]

' \nMay I have your attention please\nMay I have your attention please\nWill the real Slim Shady please stand up\nI repeat will the real Slim Shady please stand up\nWe\'re gonna have a problem here\n\n \nY\'all act like you never seen a white person before\nJaws all on the floor like Pam like Tommy just burst in the door\nAnd started whoopin\' her ass worse than before\nThey first were divorced throwin\' her over furniture Agh\nIt\'s the return of the "Oh wait no way you\'re kidding\nHe didn\'t just say what I think he did did he"\nAnd Dr Dre said… nothing you idiots\nDr Dre\'s dead he\'s locked in my basement Ha ha\nFeminist women love Eminem\n"Chicka chicka chicka Slim Shady I\'m sick of him\nLook at him walkin\' around grabbin\' his you-know-what\nFlippin\' the you-know-who" "Yeah but he\'s so cute though"\nYeah I probably got a couple of screws up in my head loose\nBut no worse than what\'s goin\' on in your parents\' bedrooms\nSometimes I wanna get on TV and just let loose\nBut ca

In [139]:
words = []
num = []
# Create list of distinct word counts per song
for i in range(0,len(superclean_df)):
    split = str(superclean_df.loc[i, 'Lyrics']).split()
    for word in split:
        if word.lower() not in words:
            words.append(word.lower())
        else:
            pass
    num.append(len(words))
    words = []
num

[363,
 407,
 227,
 179,
 198,
 200,
 191,
 249,
 93,
 140,
 105,
 225,
 175,
 115,
 184,
 716,
 308,
 221,
 132,
 310,
 163,
 207,
 190,
 167,
 214,
 277,
 183,
 264,
 237,
 576,
 55,
 37,
 209,
 84,
 261,
 153,
 23,
 102,
 470,
 38,
 114,
 105,
 38,
 73,
 98,
 11,
 71,
 71,
 36,
 101,
 131,
 1,
 242,
 1,
 154,
 430,
 1423,
 44,
 65,
 17,
 1,
 228,
 92,
 79,
 106,
 69,
 106,
 570,
 79,
 144,
 58,
 124,
 212,
 63,
 129,
 607,
 113,
 147,
 157,
 107,
 117,
 114,
 138,
 97,
 199,
 91,
 205,
 102,
 190,
 143,
 62,
 636,
 88,
 160,
 76,
 80,
 122,
 84,
 1,
 170,
 115,
 133,
 57,
 94,
 126,
 1122,
 153,
 151,
 74,
 66,
 117,
 78,
 428,
 70,
 103,
 319,
 140,
 157,
 1,
 103]

In [135]:
distinct_df = superclean_df.copy()
distinct_df['Distinct Word Count'] = num
distinct_df.head()

,Track Name,Artist,Track ID,Genres,Acousticness,Danceability,Energy,Instrumentalness,Liveness,Loudness,Speechiness,Tempo,Valence,Lyrics,Distinct Word Count
0,The Real Slim Shady,Eminem,3yfqSUWxFvZELEM4PmlwIR,1,0.0302,0.949,0.661,0.000000,0.0454,-4.244,0.0572,104.504,0.760,\nMay I have your attention please\nMay I hav...,363
1,MotorSport,Migos,4wFjTWCunQFKtukqrNijEt,1,0.0305,0.904,0.518,0.000000,0.3250,-5.320,0.1830,137.996,0.188,\nYeah woo yeah skrrt\nM-M-M-Murda\n\n \nMoto...,407
2,In Da Club,50 Cent,5Mcvgt2GbxxCp6cz3Kfjey,1,0.3250,0.906,0.663,0.000000,0.0883,-3.955,0.3550,90.042,0.847,\n\n \nGo go\nGo go go go\nGo shawty it's you...,227
3,Gangsta's Paradise,Coolio,1DIXPcTDzTj8ZMHt3PDt8p,1,0.0655,0.647,0.514,0.000000,0.3980,-10.050,0.0593,79.975,0.386,\n\n \nAs I walk through the valley of the sh...,179
4,rockstar,Post Malone,19bX4zmC2zDben2ldhoRB3,1,0.1450,0.594,0.507,0.000871,0.1320,-6.141,0.0816,159.729,0.137,\nHahahahaha\nTank God\nAyy ayy\n\n \nI've be...,198


In [140]:
#Output the dataframe to a .json
distinct_df.to_json('distinct_lyric_df.json')